# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import pandas as pd
import numpy as np 
from sklearn import datasets
import joblib

from azureml.core import Experiment, Model, Environment, ScriptRunConfig
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Dataset, Datastore 
from azureml.pipeline.steps import AutoMLStep
from azureml.core.model import InferenceConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'heart-failure-pred'

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-159680
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-159680


In [3]:
datastore_name = 'workspaceblobstore'
datastore = Datastore.get(ws, datastore_name)
datastore_path = [(datastore, 'UI/09-30-2021_102042_UTC/heart_failure_clinical_records_dataset.csv')]
ds = Dataset.Tabular.from_delimited_files(path=datastore_path)
ds = ds.take(10).to_pandas_dataframe()

In [4]:
from azureml.core.compute_target import ComputeTargetException


amlcompute_cluster_name = "project-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           min_nodes=1,    
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
import os 
import shutil 

early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2)

if "training" not in os.listdir():
    os.mkdir("./training")
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
shutil.copy('./train.py', script_folder)    

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
                                      "--C": uniform(0.1,1),
                                      "max_iter": choice([50, 75, 100, 125])
})
#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=script_folder,compute_target="project-cluster", entry_script='train.py', conda_packages= ["pandas"]
)

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4,
    estimator=estimator,
    policy=early_termination_policy

)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [6]:
#TODO: Submit your experiment
hyperdrive_run = exp.submit(config=hyperdrive_run_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_61681220-4610-4194-9fdf-e09b6a07ddf1
Web View: https://ml.azure.com/runs/HD_61681220-4610-4194-9fdf-e09b6a07ddf1?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-159680/workspaces/quick-starts-ws-159680&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-09-30T10:27:18.020486][API][INFO]Experiment created<END>\n""<START>[2021-09-30T10:27:18.634867][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-09-30T10:27:18.838996][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_61681220-4610-4194-9fdf-e09b6a07ddf1
Web View: https://ml.azure.com/runs/HD_61681220-4610-4194-9fdf-e09b6a07ddf1?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-159680/workspaces/quick-starts-ws-159680&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_61681220-4610-4194-9fdf-e09b6a07ddf1',
 'target': 'project-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-30T10:27:17.777635Z',
 'endTimeUtc': '2021-09-30T10:43:01.932844Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '14c98687-7514-4124-8024-28f61d8ea7df',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1056-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.34.0',
  'score': '0.8777777777777778',
  'best_child_run_id': 'HD_61681220-4610-4194-9fdf-e09b6a07ddf1_3',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg159680.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_61681220-4610-4194-9fdf-e09b6a07ddf1/azureml-logs/hyperdrive.txt

KeyError: 'log_files'

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_61681220-4610-4194-9fdf-e09b6a07ddf1
Web View: https://ml.azure.com/runs/HD_61681220-4610-4194-9fdf-e09b6a07ddf1?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-159680/workspaces/quick-starts-ws-159680&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_61681220-4610-4194-9fdf-e09b6a07ddf1
Web View: https://ml.azure.com/runs/HD_61681220-4610-4194-9fdf-e09b6a07ddf1?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-159680/workspaces/quick-starts-ws-159680&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_61681220-4610-4194-9fdf-e09b6a07ddf1',
 'target': 'project-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-30T10:27:17.777635Z',
 'endTimeUtc': '2021-09-30T10:43:01.932844Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '14c98687-7514-4124-8024-28f61d8ea7df',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1056-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.34.0',
  'score': '0.8777777777777778',
  'best_child_run_id': 'HD_61681220-4610-4194-9fdf-e09b6a07ddf1_3',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg159680.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_61681220-4610-4194-9fdf-e09b6a07ddf1/azureml-logs/hyperdrive.txt

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run.get_details() 

{'runId': 'HD_61681220-4610-4194-9fdf-e09b6a07ddf1_3',
 'target': 'project-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-30T10:35:12.203617Z',
 'endTimeUtc': '2021-09-30T10:35:46.487846Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '14c98687-7514-4124-8024-28f61d8ea7df',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '0.2601990759817395', '--max_iter', '100'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'project-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
 

In [9]:
#TODO: Save the best model
best_model = best_run.register_model(model_name = 'best_hf_model', model_path = 'outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [12]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

